In [46]:
import matplotlib.pyplot as plt
from collections import defaultdict
from collections import Counter
from scipy.sparse import csr_matrix, find, bmat
import numpy as np
from sklearn.cluster import KMeans
import numpy as np
import random
from sklearn.metrics.cluster import normalized_mutual_info_score
from sklearn.utils import shuffle
from sklearn.metrics import calinski_harabaz_score

In [47]:
file = 'train.dat'
data = open(file, 'r')
#print data

In [48]:
docs = list()
for row in data:
    docs.append(row.rstrip().split(" "))
print(len(docs))

8580


In [49]:
def csr_build(dataIndex, value, nnz, nrows):
    ind = np.zeros(nnz, dtype=np.int)
    val = np.zeros(nnz, dtype=np.double)
    ptr = np.zeros(nrows+1, dtype=np.int)
    i = 0
    n = 0
    
    for (d,v) in zip(dataIndex, value):
        l = len(d)
        for j in range(l):
#             print j, k
            ind[int(j) + n] = d[j]
            val[int(j) + n] = v[j]
        
        ptr[i+1] = ptr[i] + l
        n += l
        i += 1
    
    mat = csr_matrix((val, ind, ptr), shape=(nrows, max(ind)+1), dtype=np.double)
    mat.sort_indices()
    
    return mat 

# scale matrix and normalize its rows
def csr_idf(mat, copy=False, **kargs):
    r""" Scale a CSR matrix by idf. 
    Returns scaling factors as dict. If copy is True, 
    returns scaled matrix and scaling factors.
    """
    if copy is True:
        mat = mat.copy()
    nrows = mat.shape[0]
    nnz = mat.nnz
    ind, val, ptr = mat.indices, mat.data, mat.indptr
    # document frequency
    df = defaultdict(int)
    for i in ind:
        df[i] += 1
    # inverse document frequency
    for k,v in df.items():
        df[k] = np.log(nrows / float(v))  ## df turns to idf - reusing memory
    # scale by idf
    for i in range(0, nnz):
        val[i] *= df[ind[i]]
    return mat

In [50]:
def a(docs):
    # sperate indices and values
    dataIndex = list()
    value = list()
    for d in docs:
        d_index = list()
        d_value = list()
        for i in range(0,len(d),2):      
            d_index.append(d[i])
        for j in range(1,len(d),2):     
            d_value.append(d[j])
        dataIndex.append(d_index)
        value.append(d_value)
        
    nrows = len(docs)

    idx = {}
    tid = 0
    nnz = 0
    ncol = 0
    _max = list()
    for d in dataIndex:
        nnz += len(d)
        _max.append(max(d))
        for w in d:
            if w not in idx:
#             print(w)
                idx[w] = tid
                tid += 1
    mat1 = csr_build(dataIndex, value, nnz, nrows)
    mat1 = csr_idf(mat1, copy=True)
    nrows = mat1.shape[0]
    nnz = mat1.nnz
    ind, val, ptr = mat1.indices, mat1.data, mat1.indptr
    # normalize
    for i in range(nrows):
        rsum = 0.0    
        for j in range(ptr[i], ptr[i+1]):
            rsum += val[j]**2
        if rsum == 0.0:
            continue  # do not normalize empty rows
        rsum = float(1.0/np.sqrt(rsum))
        for j in range(ptr[i], ptr[i+1]):
            val[j] *= rsum
    return mat1

In [51]:
mat3 = a(docs)

In [52]:
def initCentorids(x, k):
    x_shuffle = shuffle(x, n_samples=2)
    return x_shuffle

In [53]:
def sim(x1, x2):
    print x1.shape, x2.shape
    sims = x1.dot(x2.T)
    return sims

In [54]:
def findCentroids(matc, centroids):
    idx = list()
    simsMatrix = sim(matc, centroids)

    for i in range(simsMatrix.shape[0]):
        row = simsMatrix.getrow(i).toarray()[0].ravel()
        top_indices = row.argsort()[-1]
        idx.append(top_indices + 1)
    return idx

In [55]:
def computeMeans(matm, idx, k):
    centroids = list()
    for i in range(1,k+1):
        indi = [j for j, x in enumerate(idx) if x == i]
        members = matm[indi,:]
        if (members.shape[0] > 1):
            centroids.append(members.toarray().mean(0))
    centroids_csr = csr_matrix(centroids)
    return centroids_csr

In [56]:
def kmeans(k, matk, n_iter):
    centroids = initCentorids(matk, k)
    for _ in range(n_iter): 
        idx = findCentroids(matk, centroids)
        centroids = computeMeans(matk, idx, k)        
    return idx

In [57]:
def convert_to_csr(l1):
    mcsr = list()
#    print l1
    for i in l1:
        mcsr.append(docs[i])
    return a(mcsr)
#convert_to_csr([1,2,3])

In [58]:
unselected_clusters = set()
def choose_cluster(list_of_clusters):
    global unselected_clusters
    l2 = list()
    var =[]
    for i in list_of_clusters:
        unselected_clusters.add(tuple(i))
    greatest = 0

    for i in unselected_clusters:
        l2.append(convert_to_csr(i))
#    print l2[1]
    for l in l2:
        d = np.var(l.toarray())
        var.append(d)
    
#    print  var
#   print max(var)
    
    csr_l2 = l2[(var.index(max(var)))]
    indx = list(unselected_clusters)[(var.index(max(var)))]
    unselected_clusters.remove(indx)
    return indx, csr_l2
#choose_cluster([[1,2,3],[7,8]])

In [59]:
temp = list()
def bisecting(matb):
    temp=matb.copy()
    m = [[]]
    for i in range(1, 7):
        c1 = list()
        c2 = list()
        ids = kmeans(2, temp, 8)
#         print ids
        if len(m[0]) == 0 :
            for j in range(len(ids)):
                if ids[j] == 1:
                    c1.append(j)
                else :
                    c2.append(j)
        else :
            for k in range(len(ids)):
                if ids[k] == 1:
                    c1.append(m[0][k])
                else :
                    c2.append(m[0][k])
        # m = chosen cluster indexes, m1 = not selected
        im, smat = choose_cluster([c1, c2])
        m = [im]
        # marix at the indexes of the choosen cluster "m"
        temp = smat.copy()
    for i in m:
        unselected_clusters.add(tuple(i))
    return unselected_clusters
        
print "===================================="
clusterings = bisecting(mat3)
# print clusterings

(8580, 126356) (2, 126356)
(8580, 126356) (2, 126356)
(8580, 126356) (2, 126356)
(8580, 126356) (2, 126356)
(8580, 126356) (2, 126356)
(8580, 126356) (2, 126356)
(8580, 126356) (2, 126356)
(8580, 126356) (2, 126356)
(3509, 125315) (2, 125315)
(3509, 125315) (2, 125315)
(3509, 125315) (2, 125315)
(3509, 125315) (2, 125315)
(3509, 125315) (2, 125315)
(3509, 125315) (2, 125315)
(3509, 125315) (2, 125315)
(3509, 125315) (2, 125315)
(1218, 122728) (2, 122728)
(1218, 122728) (2, 122728)
(1218, 122728) (2, 122728)
(1218, 122728) (2, 122728)
(1218, 122728) (2, 122728)
(1218, 122728) (2, 122728)
(1218, 122728) (2, 122728)
(1218, 122728) (2, 122728)
(385, 122728) (2, 122728)
(385, 122728) (2, 122728)
(385, 122728) (2, 122728)
(385, 122728) (2, 122728)
(385, 122728) (2, 122728)
(385, 122728) (2, 122728)
(385, 122728) (2, 122728)
(385, 122728) (2, 122728)
(172, 117442) (2, 117442)
(172, 117442) (2, 117442)
(172, 117442) (2, 117442)
(172, 117442) (2, 117442)
(172, 117442) (2, 117442)
(172, 117442) 

In [60]:
def zerolistmaker(n):
    listofzeros = [0] * n
    return listofzeros

# write the predictions in file
output_file = open('clusterings9-7iter.dat', 'w')
predictionslist = zerolistmaker(len(docs))
clusterIndex = 1

for n in clusterings:
    for pointIndex in n:
        predictionslist[pointIndex] = clusterIndex
    clusterIndex = clusterIndex + 1
print predictionslist


for pred in predictionslist:
    output_file.write(str(pred) + '\n');


output_file.close();

[1, 2, 1, 1, 1, 6, 6, 1, 1, 1, 1, 4, 1, 2, 1, 4, 4, 4, 1, 1, 1, 4, 4, 4, 1, 3, 4, 6, 1, 1, 1, 4, 1, 1, 1, 4, 4, 1, 1, 4, 4, 1, 1, 4, 4, 1, 1, 1, 4, 1, 1, 6, 4, 1, 1, 1, 1, 6, 4, 1, 6, 1, 6, 4, 1, 6, 4, 4, 4, 1, 4, 1, 1, 4, 1, 1, 6, 4, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 4, 4, 1, 1, 7, 1, 1, 4, 6, 1, 1, 1, 1, 1, 6, 1, 1, 1, 1, 1, 1, 1, 1, 1, 6, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 6, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 6, 2, 4, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 6, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 6, 7, 6, 4, 1, 1, 1, 4, 4, 6, 1, 4, 1, 1, 1, 1, 1, 4, 4, 1, 6, 6, 1, 1, 4, 1, 1, 1, 1, 7, 1, 4, 1, 1, 4, 4, 6, 1, 1, 1, 1, 4, 1, 4, 1, 1, 1, 3, 